> # HEAP

>Imports

In [41]:
from __future__ import annotations
from typing import Callable, Final, Optional, TypeAlias
import time, random, statistics

>Variables

In [42]:
ITERATIONS : Final[int] = 50_000
HEAP_SIZE : Final[int] = 10_000

RESULT : TypeAlias = list[tuple[str, int, float, float, float, float, float]]

>Class

In [43]:
class Heap:
    """
    A binary heap implementation supporting both min-heap and max-heap behavior.

    Attributes:
        _arr (list[int]): Internal array representing the heap.
        _is_min_heap (bool): If True, heap behaves as a min-heap; otherwise, as a max-heap.
    """
    __slots__ = ("_arr", "_is_min_heap")
    def __init__(self, arr: list[int], isMinHeap: bool = False) -> None:
        """
        Initialize a Heap from an existing list of integers.

        Args:
            arr (list[int]): Initial elements to store in the heap.
            isMinHeap (bool): Whether the heap is a min-heap (default False = max-heap).
        """
        self._arr = list(arr)
        self._is_min_heap = isMinHeap
        self.build_heap()

    # Helpers
    def _left(self, index: int) -> int:
        """
        Return the index of the left child of a node.

        Args:
            index (int): Parent node index.

        Returns:
            int: Left child index.
        """
        return (index * 2) + 1

    def _right(self, index: int) -> int:
        """
        Return the index of the right child of a node.

        Args:
            index (int): Parent node index.

        Returns:
            int: Right child index.
        """
        return (index * 2) + 2

    def _parent(self, index: int) -> int:
        """
        Return the index of the parent of a node.

        Args:
            index (int): Child node index.

        Returns:
            int: Parent node index.
        """
        return (index - 1) // 2

    def _swap(self, a: int, b: int) -> None:
        """
        Swap two elements in the heap array.

        Args:
            a (int): Index of the first element.
            b (int): Index of the second element.
        """
        self._arr[a], self._arr[b] = self._arr[b], self._arr[a]

    def _compare(self, a: int, b: int) -> bool:
        """
        Compare two elements in the heap based on heap type.

        Args:
            a (int): Index of the first element.
            b (int): Index of the second element.

        Returns:
            bool: True if element at index a has higher priority than element at index b.
        """
        return self._arr[a] < self._arr[b] if self._is_min_heap else self._arr[a] > self._arr[b]

    @property
    def size(self) -> int:
        """
        Return the number of elements in the heap.

        Returns:
            int: Heap size.
        """
        return len(self._arr)

    def is_empty(self) -> bool:
        """
        Check whether the heap is empty.

        Returns:
            bool: True if the heap is empty, False otherwise.
        """
        return len(self._arr) == 0

    def heapify_down(self, index: int, size: int) -> None:
        """
        Restore the heap property by moving an element downwards.

        Args:
            index (int): Starting index for heapification.
            size (int): Number of elements to consider in the heap.
        """
        cumulative: int = index
        left: int = self._left(index)
        right: int = self._right(index)

        if left < size and self._compare(left, cumulative):
            cumulative = left

        if right < size and self._compare(right, cumulative):
            cumulative = right

        if cumulative != index:
            self._swap(index, cumulative)
            self.heapify_down(cumulative, size)

    def heapify_up(self, index: int) -> None:
        """
        Restore the heap property by moving an element upwards.

        Args:
            index (int): Starting index for heapification.
        """
        while index > 0:
            parent = self._parent(index)
            if self._compare(index, parent):
                self._swap(index, parent)
                index = parent
            else:
                break

    def build_heap(self) -> None:
        """
        Convert the internal array into a valid heap.
        """
        start_index: int = (self.size // 2) - 1

        for i in range(start_index, -1, -1):
            self.heapify_down(i, self.size)

    def peek(self) -> Optional[int]:
        """
        Return the top element of the heap without removing it.

        Returns:
            Optional[int]: Root element of the heap, or None if empty.
        """
        return None if self.is_empty else self._arr[0]

    def push(self, value: int) -> None:
        """
        Insert a new value into the heap.

        Args:
            value (int): Value to insert.
        """
        self._arr.append(value)
        self.heapify_up(len(self._arr) - 1)

    def pop(self) -> int:
        """
        Remove and return the root element of the heap.

        Returns:
            int: The root value.

        Raises:
            IndexError: If the heap is empty.
        """
        if self.is_empty():
            raise IndexError("Pop from empty heap")

        root = self._arr[0]
        lastIndex = self.size - 1

        self._swap(0, lastIndex)
        self._arr.pop()

        if not self.is_empty():
            self.heapify_down(0, self.size)
        return root

    def push_pop(self, value: int) -> int:
        """
        Push a value onto the heap and then pop and return the root.

        This operation is more efficient than calling push() followed by pop().

        Args:
            value (int): Value to push.

        Returns:
            int: The popped value.
        """
        if self.is_empty():
            self.push(value)
            return value

        root = self._arr[0]

        if (self._is_min_heap and value > root) or not (self._is_min_heap and value < root):
            self._arr[0] = value
            self.heapify_down(0, self.size)
            return root
        else:
            return value

    def replace(self, oldValue: int, newValue: int) -> None:
        """
        Replace an existing value in the heap with a new value.

        Args:
            oldValue (int): Value to replace.
            newValue (int): New value to insert.

        Raises:
            ValueError: If the heap is empty or the value is not found.
        """
        if self.is_empty():
            raise ValueError("Heap is empty")

        try:
            index = self._arr.index(oldValue)
        except ValueError:
            raise ValueError(f"{oldValue} not found in heap")

        self._arr[index] = newValue
        parent = self._parent(index)

        if index > 0 and self._compare(index, parent):
            self.heapify_up(index)
        else:
            self.heapify_down(index, self.size)

    def clear(self) -> None:
        """
        Remove all elements from the heap.
        """
        self._arr.clear()

    def display_contents(self) -> None:
        """
        Display the heap in a tree-like level-by-level format.
        """
        if self.is_empty():
            print("Empty Heap")
            return

        print("[MinHeap]:" if self._is_min_heap else "[MaxHeap]:")

        n: int = self.size
        level: int = 0
        index: int = 0

        while index < n:
            level_size = 2 ** level
            level_nodes = self._arr[index: index + level_size]

            indent = " " * (2 ** (max(0, (self.size.bit_length() - level - 1))))
            spacing = " " * (2 ** (max(0, (self.size.bit_length() - level))))

            print(indent + spacing.join(str(v) for v in level_nodes))

            index += level_size
            level += 1

    def print_contents(self) -> None:
        """
        Print the raw internal array representing the heap.
        """
        if self.is_empty():
            print("Empty tree")
            return

        print("[MinHeap]:" if self._is_min_heap else "[MaxHeap]:")
        print(self._arr)

    def __repr__(self) -> str:
        """
        Return a string representation of the heap.
        """
        heap_type = "MinHeap" if self._is_min_heap else "MaxHeap"
        return f"[{heap_type}] {self._arr}"

    def __len__(self) -> int:
        """
        Return the number of elements in the heap.
        """
        return self.size

    def __bool__(self) -> bool:
        """
        Return True if the heap is not empty.
        """
        return not self.is_empty()

    def items(self) -> list[int]:
        """
        Return a shallow copy of the heap elements.

        Returns:
            list[int]: Heap contents.
        """
        return self._arr.copy()


In [67]:
def benchmark(func : Callable, iterations : int, *args, **kwargs) -> tuple[float, float, list[float]]:
    times = []
    for _ in range(iterations):
        start = time.perf_counter()
        func(*args, **kwargs)
        end = time.perf_counter()
        times.append(end - start)
    total_time = sum(times)
    avg_time = total_time / len(times)
    return total_time, avg_time, times

def benchmark_heap(heap_size: int = HEAP_SIZE, iterations: int = ITERATIONS, is_min_heap: bool = True) -> None:
    data = [random.randint(0, heap_size * 10) for _ in range(heap_size)]
    heap = Heap(data, isMinHeap=is_min_heap)

    #Name of op, iterations, total time, avg time, min time, max time, std devation
    results: RESULT = []

    def record(name: str, total: float, avg: float, times: list[float], iters: int):
        results.append((
            name,
            iters,
            total,
            avg,
            min(times),
            max(times),
            statistics.stdev(times) if len(times) > 1 else 0.0
    ))
        
    # push
    total, avg, times = benchmark(lambda: heap.push(random.randint(0, heap_size)), iterations)
    record("push", total, avg, times, iterations)

    # pop
    total, avg, times = benchmark(lambda: (heap.pop(), heap.push(random.randint(0, heap_size))), iterations)
    record("pop", total, avg, times, iterations)

    # peek
    total, avg, times = benchmark(lambda: heap.peek(), iterations)
    record("peek", total, avg, times, iterations)

    # push_pop
    total, avg, times = benchmark(lambda: heap.push_pop(random.randint(0, heap_size)), iterations)
    record("push_pop", total, avg, times ,iterations)

    # replace
    total, avg, times = benchmark(lambda: heap.replace(heap._arr[random.randint(0, heap.size - 1)],random.randint(0, heap_size)), iterations)
    record("replace", total, avg, times, iterations)

    # build_heap 
    total, avg, times = benchmark(lambda: Heap(data, isMinHeap=is_min_heap), iterations)
    record("build_heap", total, avg, times, iterations)

    print_table(results)


def print_table(results: RESULT) -> None:
    print(f"{'Operation':<25} {'Iterations':>10} {'Total(s)':>10} {'Avg(s)':>10} {'Min(s)':>10} {'Max(s)':>10} {'StdDev':>10}")
    print("-"*90)
    for op, count, total, avg, mn, mx, stdev in results:
        print(f"{op:<25} {count:>10} {total:>10.4f} {avg:>10.8f} {mn:>10.8f} {mx:>10.8f} {stdev:>10.8f}")

In [76]:
if __name__ == "__main__":
    benchmark_heap(heap_size=1_000_000, iterations=1_000, is_min_heap=True)


Operation                 Iterations   Total(s)     Avg(s)     Min(s)     Max(s)     StdDev
------------------------------------------------------------------------------------------
push                            1000     0.0024 0.00000237 0.00000060 0.00101530 0.00003207
pop                             1000     0.0093 0.00000928 0.00000430 0.00003950 0.00000230
peek                            1000     0.0001 0.00000015 0.00000010 0.00000170 0.00000007
push_pop                        1000     0.0082 0.00000818 0.00000060 0.00002620 0.00000172
replace                         1000    13.9319 0.01393187 0.00010060 0.03169320 0.00626824
build_heap                      1000   588.0408 0.58804078 0.50645580 1.02114870 0.03644119
